In [1]:
from abc import ABC, abstractmethod

## Abstract query expander class
All query models will inherit this class. The class stores previous conversation history for each topic, which may be useful for many query models. The class method get_query will be called in evaluation scripts. All query models must inherit the generated_query method, which is called within the get_query method

In [2]:
class QueryExpander(ABC):
    
    def __init__(self):
        self.question_history = {}
        self.query_history = {}
        
    def get_query(self, query, depth, topic_num):        
        generated_query = self.generate_query(query, depth, topic_num)
        
        if topic_num not in self.question_history:
            self.question_history[topic_num] = []
            self.query_history = []
        self.question_history[topic_num].append(query)
        self.query_history.append(generated_query)
        return generated_query
        
    @abstractmethod
    def generate_query(self, query, depth, topic_num):
        pass

## Example class
ExampleQuery is an example class that inherits the QueryExpander class and generate_query method. The class will print all available topic data each time it "generates" a new query (previous questions and generated queries)

In [3]:
class ExampleQuery(QueryExpander):
    
    def __init__(self):
        super().__init__()
        
    def generate_query(self, query, depth, topic_num):
        print(f'query: {query}\ndepth: {depth}\ntopic_num: {topic_num}')
        print("Question history ", self.question_history)
        print("Query history ", self.query_history)
        return f'generated query topic {topic_num} depth {depth}' 

In [4]:
qe = ExampleQuery()

#### First question within the first topic
There is no available information from previous questions as this is the first one

In [5]:
qe.get_query('Do you have any Nike shoes here?', depth=1, topic_num=1)

query: Do you have any Nike shoes here?
depth: 1
topic_num: 1
Question history  {}
Query history  {}


'generated query topic 1 depth 1'

#### Second question within the first topic
We can now access and utilize the first question when generating this query

In [6]:
qe.get_query('What colors are they in', depth=2, topic_num=1)

query: What colors are they in
depth: 2
topic_num: 1
Question history  {1: ['Do you have any Nike shoes here?']}
Query history  ['generated query topic 1 depth 1']


'generated query topic 1 depth 2'

#### First question within the second topic
There is no available information from previous questions within this topic (2). The entire conversation in topic 1 is still available, but that is not useful when a topic is changed

In [7]:
qe.get_query('What is the capital of Spain?', depth=1, topic_num=2)

query: What is the capital of Spain?
depth: 1
topic_num: 2
Question history  {1: ['Do you have any Nike shoes here?', 'What colors are they in']}
Query history  ['generated query topic 1 depth 1', 'generated query topic 1 depth 2']


'generated query topic 2 depth 1'

#### Second question within the second topic
We can now access the first question from topic 2 to generate the second query 

In [8]:
qe.get_query('How many people live there?', depth=2, topic_num=2)

query: How many people live there?
depth: 2
topic_num: 2
Question history  {1: ['Do you have any Nike shoes here?', 'What colors are they in'], 2: ['What is the capital of Spain?']}
Query history  ['generated query topic 2 depth 1']


'generated query topic 2 depth 2'